In [4]:
import os
from os import listdir
from os.path import isfile, join


#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.pyplot import figure, imshow, axis
from matplotlib.image import imread

import numpy as np
import cv2
from glob import glob
import PIL.Image
from IPython.display import Image, HTML, display
import matplotlib.colors
import colorsys

# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

%matplotlib inline


In [5]:
#All functions used by the pipeline

import math
    
def bgr2rgb(inputImg):
    b,g,r = cv2.split(inputImg)       # get b,g,r
    rgb_img = cv2.merge([r,g,b])
    return rgb_img

def rgb2bgr(inputImg):
    r,g,b = cv2.split(inputImg)       # get r,g,b
    bgr_img = cv2.merge([b,g,r])
    return bgr_img

    
def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def draw_lines(img, lines, frame_height, frame_width, color=[255, 0, 0], thickness=5):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
     
    for line in lines:
        for x1,y1,x2,y2 in line:
            slope = (y2 - y1) / (x2 - x1)
            if(slope != 0):
                y_fullExt1 = int(frame_height)
                x_fullExt1 = int(((y_fullExt1 - y1)/slope) + x1)
            
                y_fullExt2 = int(0.6 * frame_height)
                x_fullExt2 = int(((y_fullExt2 - y1)/slope) + x1)
            
                cv2.line(img, (x_fullExt1, y_fullExt1), (x_fullExt2, y_fullExt2), color, thickness)
                
            
                
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    
    frame_height, frame_width = img.shape[:2]
    
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines, frame_height, frame_width)
    return line_img

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)


def displayImgs(ImagesArr):
    fig = plt.figure(figsize=(60, 60))  # width, height in inches

    for i in range(len(ImagesArr)):
        sub = fig.add_subplot(4, 2, i + 1)
        sub.imshow(ImagesArr[i], interpolation='nearest')

def rgb2hls(ImagesArr):
    hls = np.zeros_like(ImagesArr)
    hls = cv2.cvtColor(ImagesArr, cv2.COLOR_RGB2HLS)
    return hls

def rgb2hsv(ImagesArr):
    hsv = np.zeros_like(ImagesArr)
    for i in range (len(ImagesArr)):
        hsv[i] = cv2.cvtColor(Test_images[i], cv2.COLOR_RGB2HSV)
    return hsv

def yellowExtract(ImagesArr, lower_yellow, upper_yellow):

    yellowDetect = np.zeros_like(ImagesArr)
    
    yellowMask = cv2.inRange(ImagesArr, lower_yellow, upper_yellow)
    return yellowMask        

def whiteExtract(ImagesArr, lower_white, upper_white):
    
    whiteDetect = np.zeros_like(ImagesArr)
    
    whiteMask = cv2.inRange(ImagesArr, lower_white, upper_white)
    return whiteMask   

def ROI(ImagesArr, vertices):
    
    masked_imgs = np.zeros_like(ImagesArr)
    
    
    masked_imgs = region_of_interest(ImagesArr, np.int32([vertices]))
    return masked_imgs

def EdgeDetect(ImagesArr):

    Edges = canny(ImagesArr, 50, 250)
    BlurredImg = gaussian_blur(Edges, 7)
    
    return BlurredImg

def LaneDetect(ImagesArr):
    
    line_img = np.zeros_like(ImagesArr)
    Lines_img = np.uint8(line_img)
    
    #for i in range (len(BlurredImg)):
    Lines_img = hough_lines(ImagesArr , rho, theta, threshold, min_line_len, max_line_gap)

    return Lines_img

def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)
    hls = np.zeros_like(image)
    yellowMask = np.zeros_like(image)
    whiteMask = np.zeros_like(image)
    yellowDetect = np.zeros_like(image)
    whiteDetect = np.zeros_like(image)
    finalColorMask = np.zeros_like(image)
    colorMaskedimg = np.zeros_like(image)
    masked_imgs = np.zeros_like(image)
    blurredImg = np.zeros_like(image)
    Lines_img = np.uint8(image)
    lanes_frame = np.zeros_like(image)
    
    frame_height, frame_width = image.shape[:2]
    
    vertices = np.array([[(0.11*frame_width), frame_height], [frame_width ,frame_height], [(0.50*frame_width),(0.58*frame_height)], [(0.48*frame_width),(0.58*frame_height)]])
   
    hls = rgb2hls(image)
    
    yellowMask = yellowExtract(hls, lower_yellow_hls, upper_yellow_hls)
    
    whiteMask = whiteExtract(hls, lower_white_hls, upper_white_hls)
    
    
    finalColorMask = cv2.bitwise_xor(yellowMask, whiteMask)
    
    colorMaskedimg = cv2.bitwise_and(hls, hls, mask = finalColorMask)
    
    
    masked_imgs = ROI(colorMaskedimg, vertices)
    
    blurredImg = EdgeDetect(masked_imgs)
    
    Lines_img = LaneDetect(blurredImg)
    lanes_frame = weighted_img(Lines_img, image, α=0.8, β=1., λ=0.)
    return lanes_frame

In [6]:
## Tune the parameters of the pipeline functions
lower_yellow_hls = np.array([10 , 0 ,100], dtype=np.uint8)
upper_yellow_hls = np.array([40, 255 , 255], dtype=np.uint8)

lower_white_hls = np.array([0,220,0], dtype=np.uint8)
upper_white_hls = np.array([255,255,255], dtype=np.uint8)

rho = 1
theta = np.pi/180
threshold = 50 #1
min_line_len = 100 #10
max_line_gap = 160 #1

white_output = 'test_videos_output/solidWhiteRight.mp4'
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
challenge_output = 'test_videos_output/challenge_clip.mp4'

## Run the pipelin on each of the test videos
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
clip2 = VideoFileClip("test_videos/solidYellowLeft.mp4").subclip(0,5)
clip2 = VideoFileClip("test_videos/solidYellowLeft.mp4")
clip3 = VideoFileClip("test_videos/challenge.mp4").subclip(0,5)
clip3 = VideoFileClip("test_videos/challenge.mp4")

white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

yellow_clip = clip2.fl_image(process_image) #NOTE: this function expects color images!!
%time yellow_clip.write_videofile(yellow_output, audio=False)

challenge_clip = clip3.fl_image(process_image) #NOTE: this function expects color images!!
%time challenge_clip.write_videofile(challenge_output, audio=False)


[MoviePy] >>>> Building video test_videos_output/solidWhiteRight.mp4
[MoviePy] Writing video test_videos_output/solidWhiteRight.mp4


100%|█████████▉| 221/222 [00:08<00:00, 24.95it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidWhiteRight.mp4 

CPU times: user 20.4 s, sys: 220 ms, total: 20.7 s
Wall time: 9.55 s
[MoviePy] >>>> Building video test_videos_output/solidYellowLeft.mp4
[MoviePy] Writing video test_videos_output/solidYellowLeft.mp4


100%|█████████▉| 681/682 [00:30<00:00, 22.48it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidYellowLeft.mp4 

CPU times: user 1min 6s, sys: 564 ms, total: 1min 6s
Wall time: 31 s
[MoviePy] >>>> Building video test_videos_output/challenge_clip.mp4
[MoviePy] Writing video test_videos_output/challenge_clip.mp4


100%|██████████| 251/251 [00:18<00:00, 13.13it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/challenge_clip.mp4 

CPU times: user 30.3 s, sys: 1.03 s, total: 31.3 s
Wall time: 20.1 s
